<a href="https://colab.research.google.com/github/mashyko/NLP_Mecab/blob/master/Gensim_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gensimを用いた文書類似度の判定：Livedoor Newsの使用

$wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

上記サイトのlivedoor newsのデータ文書をダウンロードします。


In [0]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
!tar -zxvf ldcc-20140209.tar.gz

上記サイトのlivedoor newsから得たデータ文書をもとに作成した訓練用と検証用のデータがdataset_train.csv、 dataset_valid.csvです。

以下のコードは、livedoor_text 文書をクラス分けして df_texts (DataFrame 形式で)の中に納めています。このファイルは'class' と’body'という名前の列を持っています。'class' には classes の名前が入ります。'body' に、文章が入ります。csv 形式で保存します。このファイルの内容を知りたいときは、例えば、200行から210行前の内容を表示させたいときは 'print (df_texts[200:211])'と入力します。

In [12]:
import pandas as pd
import glob
import os

classes = ["dokujo-tsushin","it-life-hack","kaden-channel","livedoor-homme",
           "movie-enter","peachy","smax","sports-watch","topic-news"]

df_texts = pd.DataFrame(columns=['class','body'])

if __name__ == '__main__':

    # テキストの抽出
    for c in classes:
        filepath = './text/' + c + '/*.txt'
        files = glob.glob(filepath)

        for i in files:
            f = open(i)
            text = f.read()
            text = text.replace("\u3000","")
            f.close()
            row = pd.Series([c, "".join(text.split("\n")[3:])], index=df_texts.columns)
            df_texts = df_texts.append(row, ignore_index=True)

        # トレーニング、バリデーションで 8:2 に分割
    df_train = pd.DataFrame(columns=['class','body'])
    df_validation = pd.DataFrame(columns=['class','body'])

    for c in classes:
        df_text_c = df_texts[df_texts['class'] == c]

        df_text_c_train = df_text_c.head(round(len(df_text_c)*0.8))
        df_train = df_train.append(df_text_c_train, ignore_index=True)

        df_text_c_validation = df_text_c.tail(round(len(df_text_c)*0.2))
        df_validation = df_validation.append(df_text_c_validation, ignore_index=True)

    # テキストの保存
    df_train.to_csv('dataset_train.csv')
    df_validation.to_csv('dataset_validation.csv')
print (df_texts[200:210])



              class                                               body
200  dokujo-tsushin  「腐女子」という言葉をご存知でしょうか。いわゆる漫画やアニメキャラなどの男性同士の恋愛（ボー...
201  dokujo-tsushin  2010年も残りわずか。 今回は、今年1年を振り返って女性たちの「エラかったね！よくやった！...
202  dokujo-tsushin  一生のうちでこんなにも人を愛し、愛されることができたとしたら、その後の人生は一人でも生きてい...
203  dokujo-tsushin  ここ数年、美容整形に対する意識は、その手軽さからだいぶ変わってきたのではないだろうか。例えば...
204  dokujo-tsushin  2012年の幕開けと同時に恒例の冬のセールもスタート。早速セールに出かけて「昨年から狙ってい...
205  dokujo-tsushin  婚活においても、出会いの場と注目されている同窓会。同窓会をきっかけに始まる恋愛を「同窓会恋愛...
206  dokujo-tsushin  日本中、小さな街にもひとつくらいコンビニがある。いつでも気軽に欲しいものが手に入るコンビニは...
207  dokujo-tsushin  今月もこんにちは、独女の皆様。 お下劣、毒舌、支離滅裂！ドロンジョーヌ恩田です。 夏を迎え、...
208  dokujo-tsushin  年下の彼・和也さん（28歳）からの熱烈なアプローチで結婚した美也子さん（32歳）。幸せな毎日...
209  dokujo-tsushin  1月も後半に入り、引き続き厳しい寒さが続く中で「早く暖かくならないかなぁ」なんてついつぶやい...


単語の分かち書き方法

文章を単語に分かち書きするにはMeCabを使います。以下のクラスを作成し用いています。 今回は特徴を拾いやすくするために名詞だけ抽出してみることにしました。係り受けなどは考慮しないので、形容詞、副詞、動詞などは今回は含めていません。辞書は neologd を使っています。

In [13]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.


In [14]:
!mecab-config --dicdir

/usr/lib/x86_64-linux-gnu/mecab/dic


In [0]:
import sys
import MeCab
import unicodedata

class mecab_split():
    """ mecabで分かち書きなどを行う処理まとめ """
    def __init__(self):
        pass

    @staticmethod
    def split(text):
        #文字コード変換処理。変換しないと濁点と半濁点が分離する。
        text = unicodedata.normalize('NFC', text)

        result = []
        tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
        tagger.parse('') #parseToNode前に一度parseしておくとsurfaceの読み取りエラーを回避できる

        nodes = tagger.parseToNode(text)
        while nodes:
            if nodes.feature.split(',')[0] in ['名詞']:
                word = nodes.surface
                result.append(word)
            nodes = nodes.next
        return ' '.join(result)

自然言語モデルの作成:Doc2Vec でモデルを作成する

gensim というライブラリに Doc2Vec が実装されているのでそれを使います。手法は dmpv という手法を用います。 この手法で学習させる際には文書idをタグとして持つので、以下のように書きます。

In [16]:
!pip install gensim

In [0]:
import sys
import os
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

sys.path.append(os.pardir)
#from lib.mecab_split import mecab_split as ms
ms = mecab_split()

dataset_file = 'dataset_train.csv'
#dataset_file = '/content/drive/My Drive/NLP/livedoor-text/dataset_train.csv'

df_text = pd.read_csv(dataset_file)
trainings = [TaggedDocument(words = ms.split(body), tags = [i])
             for i, body in enumerate(df_text['body'])]

In [18]:
# モデルの学習, dmpvで学習させる
model = Doc2Vec(documents=trainings, dm=1, size=300, window=5, min_count=5)

# モデルの保存
#model.save('/content/drive/My Drive/NLP/livedoor-text/doc2vec.model')


/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [19]:
#model = Doc2Vec.load('/content/drive/My Drive/NLP/livedoor-text/doc2vec.model')
model.docvecs.most_similar(500)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(578, 0.8063284158706665),
 (526, 0.7901571989059448),
 (123, 0.7558071613311768),
 (515, 0.7411441206932068),
 (347, 0.7408869862556458),
 (2154, 0.7338988184928894),
 (371, 0.7334645390510559),
 (11, 0.7311739921569824),
 (938, 0.7248408198356628),
 (1121, 0.7241316437721252)]

In [20]:
model.docvecs.similarity(1,307)
#文書1と文書307の間の類似度

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.4200067

学習したモデルを使って、新規に与えた文書の同士の類似度を調べる

In [21]:
#例えば、以下の４つの新規文書の、いくつかの組み合わせの類似度を計算してみる
doc_words1 = ["ラスト", "展開" ,"早い" ,"他" ,"作品", "衝撃", "受ける" ,"裏の裏" ,"つく", "トリック" ,"毎度" ,"こと", "脱帽", "する", "読む", "やすい" ,"め" ,"ミステリー"]
doc_words2 = [ "イニシエーション・ラブ", "同様" ,"最後", "数行", "どんでん返し", "いく", "時", "時", "様々", "シーン", "する" ,"れる", "伏線" ,"散りばめる", "られる" ,"いる", "こと", "気づく"]
doc_words3 = ["ラスト", "展開" ,"早い" ,"他" ,"作品", "衝撃", "受ける" ,"裏の裏" ,"つく","ミステリー"]
doc_words4 = ["独特", "世界観", "日常" ,"離れる","落ち着く","時","読む","本"]

print ("1-2 similarity")
sim_value = model.docvecs.similarity_unseen_docs(m, doc_words1, doc_words2, alpha=1, min_alpha=0.0001, steps=5)
print (sim_value)

print ("1-3 sim")
print (model.docvecs.similarity_unseen_docs(m, doc_words1, doc_words3, alpha=1, min_alpha=0.0001, steps=5))

print ("1-4 sim")
print (model.docvecs.similarity_unseen_docs(m, doc_words1, doc_words4, alpha=1, min_alpha=0.0001, steps=5))

print ("2-3 sim")
print (model.docvecs.similarity_unseen_docs(m, doc_words2, doc_words3, alpha=1, min_alpha=0.0001, steps=5))


1-2 similarity
0.21265341
1-3 sim
0.7133255
1-4 sim
0.43777364
2-3 sim
0.42022422


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


以上
